In [1]:
import math;

In [2]:
def RWH_hierarchy(input_links, is_weighted = True, alpha_parameter = 1.0, lambda_value = 4.0, 
                  precision = 0.0000001, max_num_rounds = 10000, silent = True, check_params = True):
    """Calculates the random walk hierarchy measure introduced in 
       http://dx.doi.org/10.1038/srep17994
       Returns the RWH measure and the stationary distribution of the random walkers on the nodes, sorted
       according to the density value from largest density to lowest. The results are returned in a 
       dictionary {'RWH': float, 'p_stat': [float,str]} or {'RWH': float, 'p_stat': [float,int]}, 
       depending on the node type in the input_links.
       
       The parameters:
        -input_links: the link list of the network, [['source','target']] or [['source','target',weight]].
        -is_weighted: if True, the input_links is assumed to be weighted, otherwise input_links is assumed
                      to be unweighted.
        -alpha_parameter: a generalised decay parameter, it is advised to stick to the default value 1.0
        -lambda_value: the characteristic distance of the decaying random walk. It is advised to stick to
                       the default value 4.0
        -precision: the parameter controlling the precision of the stationarity of the walker density.
        -max_num_rounds: the maximum number of allowed iterations during the calculation of the stationary
                      walker density.
        -silent: if False, the function prints various messages about the state of the calculation.
        -chek_params: bool, if True, the types of the parameters are checked."""
    if check_params:
        Type_check_RWH_hierarchy(alpha_parameter,lambda_value,precision,max_num_rounds,is_weighted,input_links);
    f_value = math.exp(1.0/lambda_value)-1.0;
    t_in_neighs = [];
    t_out_neighs = [];
    name_id_dict = {};
    rev_dict = {};
    Prepare_neigh_list_dicts(t_in_neighs,t_out_neighs,name_id_dict,rev_dict,is_weighted,input_links);
    if not silent:
        print('RWH_hierarchy: neig. lists prepared OK.');
    T_matr = Prepare_T_matrix(t_in_neighs,t_out_neighs,alpha_parameter);
    if not silent:
        print('RWH_hierarchy: T matrix prepared OK.');
    p_stat_vec = [1.0/len(t_out_neighs)]*len(t_out_neighs);
    abs_diff = 10*precision;
    num_rounds = 0;
    if not silent:
        print('RWH_hierarchy: Calculating the stationary distribution of the walkers...')
    while(abs_diff > precision and num_rounds < max_num_rounds):
        if not silent:
            print(' iteration no. ',num_rounds);
        new_p_stat_vec = [];
        new_p_stat_vec.clear();
        new_p_stat_vec = Mult_T_matr(T_matrix = T_matr, p_stat = p_stat_vec, f_parameter = f_value);
        abs_diff = Compare_p_stats(new_p_stat_vec,p_stat_vec);
        p_stat_vec.clear();
        p_stat_vec = list(new_p_stat_vec);
        num_rounds += 1;
    if not silent:
        print('RWH_hierarchy: stationary distribution converged.');
    ordered_p_stat = Order_p_stat(p_stat_vec);
    RWH_result = Calc_RWH_from_p_stat(p_stat_vec);
    if not silent:
        print('RWH_hierarchy: final result for RWH = ',RWH_result);
    output_p = [];
    for i in range(0,len(ordered_p_stat)):
        output_p.append([ordered_p_stat[i][0],rev_dict[ordered_p_stat[i][1]]]);
    return {'RWH': RWH_result,'p_stat': output_p};

In [3]:
def Type_check_RWH_hierarchy(alpha_parameter,lambda_value,precision,max_num_rounds,is_weighted,input_links):
    if not isinstance(alpha_parameter,float):
        raise TypeError('ERROR: Type_check_RWH_hierarchy: alpha_parameter must be a float.\n');
    if not isinstance(lambda_value,float):
        raise TypeError('ERROR: Type_check_RWH_hierarchy: lambda_value must be a float.\n');
    if not isinstance(precision,float):
        raise TypeError('ERROR: Type_check_RWH_hierarchy: precision must be a float.\n');
    if not isinstance(max_num_rounds,int):
        raise TypeError('ERROR: Type_check_RWH_hierarchy: max_num_rouncs must be an int.\n');
    Type_check_input_links(is_weighted,input_links);

In [4]:
def Type_check_input_links(is_weighted,input_ls):
    if not isinstance(input_ls,list):
        raise TypeError('ERROR: Type_check_input_links: input_ls must be a list.\n');
    if len(input_ls) == 0:
        raise TypeError('ERROR: Type_check_input_links: input_ls is empty.');
    if not isinstance(is_weighted,bool):
        raise TypeError('ERROR: Type_check_input_links: is_weighted must be a bool.\n');
    if is_weighted:
        for i in range(0,len(input_ls)):
            orig_len = len(input_ls[i]);
            if orig_len < 3 :
                raise TypeError('ERROR: Type_check_input_links: entry '+str(i)+
                                ' in input_ls has length '+str(orig_len)+' instead of 3 or larger.\n');
            or_src = input_ls[i][0];
            or_trg = input_ls[i][1];
            or_wei = input_ls[i][2];
            if not (isinstance(or_src,int) or isinstance(or_src,str)):
                raise TypeError('ERROR: Type_check_input_links: source node type not int or str in entry '+str(i)+
                                ' in input_ls.\n');
            if not (isinstance(or_trg,int) or isinstance(or_trg,str)):
                raise TypeError('ERROR: Type_check_input_links: target node type not int or str in entry '+str(i)+
                               ' in input_ls.\n');  
            if not (isinstance(or_wei,float) or isinstance(or_wei,int)):
                raise TypeError('ERROR: Type_check_input_links: link weight type not float or int in entry '+str(i)+
                               ' in input_ls.\n');
    else:
        for i in range(0,len(input_ls)):
            orig_len = len(input_ls[i]);
            if orig_len < 2 :
                raise TypeError('ERROR: Type_check_input_links: entry '+str(i)+
                                ' in input_ls has length '+str(orig_len)+' instead of 2 or larger.\n');
            or_src = input_ls[i][0];
            or_trg = input_ls[i][1];
            if not (isinstance(or_src,int) or isinstance(or_src,str)):
                raise TypeError('ERROR: Type_check_input_links: source node type not int or str in entry '+str(i)+
                                ' in input_ls.\n');
            if not (isinstance(or_trg,int) or isinstance(or_trg,str)):
                raise TypeError('ERROR: Type_check_input_links: target node type not int or str in entry '+str(i)+
                               ' in input_ls.\n');  

In [5]:
def Prepare_neigh_list_dicts(in_neighs,out_neighs,name_id_dict,id_rev_dict,is_weighted,input_ls):
    """Prepares the in-neighbour and out-neighbour lists for the nodes. 
       in_neighs and out_neighs: N long list of dicts, [{neigh_id,weight}], 
       name_id_dict: {original_node_name,id},
       id_rev_dict: {id,original_node_name},
       is_weighted: bool, if True, input_ls is assumed to be weighted,
       input_ls: the input, assumed to be a list of links [['source','target']] or 
                    [['source','target',weight]] if the links are weighted."""
    in_neighs.clear();
    out_neighs.clear();
    name_id_dict.clear();
    id_rev_dict.clear();
    for i in range(len(input_ls)):
        source = input_ls[i][0];
        target = input_ls[i][1];
        weight = 1.0;
        if is_weighted:
            weight = input_ls[i][2];
        if not (source in name_id_dict):
            source_id = len(name_id_dict);
            name_id_dict[source] = source_id;
            id_rev_dict[source_id] = source;
        if not (target in name_id_dict):
            target_id = len(name_id_dict);
            name_id_dict[target] = target_id;
            id_rev_dict[target_id] = target;
        s_id = name_id_dict[source];
        t_id = name_id_dict[target];
        if s_id < len(out_neighs):
            out_neighs[s_id][t_id] = weight;
        else:
            s_ldict = {};
            s_ldict.clear();
            s_ldict[t_id] = weight;
            out_neighs.append(s_ldict);
        if s_id >= len(in_neighs):
            s_ldict = {};
            s_ldict.clear();
            in_neighs.append(s_ldict);
        if t_id < len(in_neighs):
            in_neighs[t_id][s_id] = weight;
        else:
            t_ldict = {};
            t_ldict.clear();
            t_ldict[s_id] = weight;
            in_neighs.append(t_ldict);
        if t_id >= len(out_neighs):
            t_ldict = {};
            t_ldict.clear();
            out_neighs.append(t_ldict);

In [6]:
def Prepare_T_matrix(in_neighs,out_neighs,alpha_param):
    T_matrix = [];
    T_matrix.clear();
    T_column_sum = [0]*len(out_neighs);
    for i in range(0,len(out_neighs)):
        row = {};
        row.clear();
        sum_i_out = 0;
        for out_n_ind in out_neighs[i]:
            sum_i_out += out_neighs[i][out_n_ind];
        for out_n_j in out_neighs[i]:
            link_weight = out_neighs[i][out_n_j];
            sum_j_in = 0;
            for in_j in in_neighs[out_n_j]:
                sum_j_in += in_neighs[out_n_j][in_j];
            row[out_n_j] = link_weight/sum_j_in*((link_weight/sum_i_out)**alpha_param);
            T_column_sum[out_n_j] += row[out_n_j];
        T_matrix.append(row);
    for j in range(0,len(T_column_sum)):
        if T_column_sum[j] < 1.0:
            T_matrix[j][j] = 1.0 - T_column_sum[j];
    return T_matrix;

In [7]:
def Mult_T_matr(T_matrix,p_stat,f_parameter):
    new_vec = list();
    new_vec.clear();
    new_rand_walker_ins = f_parameter/float(len(p_stat));
    for i in range (0,len(T_matrix)):
        if len(T_matrix[i]) > 0 :
            T_i_sum = 0;
            for T_ij in T_matrix[i]:
                T_i_sum += T_matrix[i][T_ij]*(p_stat[T_ij]+new_rand_walker_ins);
            new_vec.append(T_i_sum);
        else :
            new_vec.append(0.0);
    sum_vec = sum(new_vec);
    for i in range(0,len(new_vec)):
        new_vec[i] /= sum_vec;
    return new_vec;

In [8]:
def Compare_p_stats(new_p_stat,old_p_stat):
    sum_diff = 0;
    for i in range(0,len(new_p_stat)):
        sum_diff += math.fabs(new_p_stat[i]-old_p_stat[i]);
    return sum_diff;

In [9]:
def Order_p_stat(p_stat_vec):
    p_dict = {};
    p_dict.clear();
    for i in range(0,len(p_stat_vec)):
        p_dict[p_stat_vec[i]] = i;
    return sorted(p_dict.items(), key = lambda x:x[0], reverse = True);

In [10]:
def Calc_RWH_from_p_stat(p_stat_result):
    sum = 0.0;
    for i in range(0,len(p_stat_result)):
        sum += p_stat_result[i]*p_stat_result[i];
    return math.sqrt(sum*len(p_stat_result)-1.0);